## A script to populate the list of files in a specific class given that they're in a seperate file

In [1]:
from pathlib import Path

def populate(path, train_images, labels, label):
    directory = Path(path)
    # Loop through all files in the directory
    i = 0
    for file in directory.glob("*.jpg"):
        train_images.append(path + "/" + file.name)
        labels.append(label)
        i += 1

## populating our training lists

In [2]:
PLATE = 1
NO_PLATE = 0

train_images = []
train_labels = []

populate("plate", train_images, train_labels, PLATE)
populate("no-plate", train_images, train_labels, NO_PLATE)

In [3]:
import cv2
from skimage.feature import hog

hog_features = []
for img_path in train_images:
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, None, fx=0.5, fy=0.5)
    features = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), block_norm='L2-Hys',
                   transform_sqrt=True, feature_vector=True)
    hog_features.append(features)

## train the model and check accuracy

In [4]:
from sklearn.model_selection import train_test_split
from sklearn import svm

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(hog_features, train_labels, test_size=0.2, random_state=42)

# Train the SVM model on the training set
svm_model = svm.SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Evaluate the SVM model on the testing set
accuracy = svm_model.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.8928571428571429
